<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/MultiTimeAnalysis_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install pandas
!pip install matplotlib
!pip install datetime

In [ ]:
# 데이터 수집 모듈
def collect_data(symbol, start_date, end_date, output_file='/content/drive/MyDrive/US MARKET/Data/meta_1112.xlsx'):
    import yfinance as yf
    import pandas as pd

    # Meta (META) 주식 데이터를 yfinance를 통해 다운로드
    df = yf.download(symbol, start=start_date, end=end_date)

    # 데이터 확인을 위해 데이터프레임 프린트
    print(df.head())

    # 열 이름 변경 (영어로 유지)
    df.reset_index(inplace=True)
    df = df.rename(columns={'Date': 'Date', 'Open': 'Open', 'High': 'High', 'Low': 'Low', 'Close': 'Close', 'Volume': 'Volume'})

    # 날짜 형식 변경 및 데이터 정렬
    df['Date'] = pd.to_datetime(df['Date']).tz_localize(None)  # tz 정보 제거하여 tz-naive로 변경

    # 데이터 저장
    df.to_excel(output_file, index=False)
    print(f"데이터가 {output_file} 파일에 저장되었습니다.")

# 데이터 수집 모듈 실행
collect_data('META', '2020-01-01', '2024-11-12')

# 이후 모듈에서 데이터 분석 시작
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Excel 파일로부터 데이터 로드
df = pd.read_excel('/content/drive/MyDrive/US MARKET/Data/meta_1112.xlsx')

# 특정 시점 분석 날짜 설정
analysis_date = pd.to_datetime('2024-11-12')

# 날짜 필터 적용
df = df[df['Date'] <= analysis_date]

# 필요한 열이 모두 존재하는지 확인
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in required_columns:
    if col not in df.columns:
        raise KeyError(f"필수 열 '{col}'이(가) 데이터프레임에 존재하지 않습니다.")

# 일 단위, 주 단위, 월 단위 데이터 리샘플링
# 일 단위 데이터는 그대로 사용하고 주 단위와 월 단위로 리샘플링

# 일 단위 데이터
daily_df = df.copy()

# 주 단위 데이터 리샘플링
weekly_df = df.set_index('Date').resample('W-MON').agg({'Open': 'first', 'Close': 'last', 'High': 'max', 'Low': 'min', 'Volume': 'sum'}).dropna()

# 월 단위 데이터 리샘플링
monthly_df = df.set_index('Date').resample('M').agg({'Open': 'first', 'Close': 'last', 'High': 'max', 'Low': 'min', 'Volume': 'sum'}).dropna()

# 각 데이터프레임에서 이동평균 계산
for dataset, name in zip([daily_df, weekly_df, monthly_df], ['Daily', 'Weekly', 'Monthly']):
    dataset['MA_10'] = dataset['Close'].rolling(window=10).mean()
    dataset['MA_50'] = dataset['Close'].rolling(window=50).mean()
    print(f"{name} 이동평균 계산 완료.")

# 다중 시점 포지션 분석
# 1. 일 단위 데이터 분석
print("\n[Daily Analysis]")
daily_last_row = daily_df.iloc[-1]
if daily_last_row['Close'] > daily_last_row['MA_10'] and daily_last_row['Close'] > daily_last_row['MA_50']:
    print(f"현재 주가는 단기(10일) 및 장기(50일) 이동평균선 위에 있어 상승 추세를 보입니다.")
else:
    print(f"현재 주가는 이동평균선 아래에 위치하여 하락 추세 가능성을 나타냅니다.")

# 2. 주 단위 데이터 분석
print("\n[Weekly Analysis]")
weekly_last_row = weekly_df.iloc[-1]
if weekly_last_row['Close'] > weekly_last_row['MA_10'] and weekly_last_row['Close'] > weekly_last_row['MA_50']:
    print(f"주 단위 이동평균선 분석 결과, 주가가 상승 추세에 있습니다.")
else:
    print(f"주 단위 이동평균선 분석 결과, 주가가 하락 추세에 있을 수 있습니다.")

# 3. 월 단위 데이터 분석
print("\n[Monthly Analysis]")
monthly_last_row = monthly_df.iloc[-1]
if monthly_last_row['Close'] > monthly_last_row['MA_10'] and monthly_last_row['Close'] > monthly_last_row['MA_50']:
    print(f"월 단위 이동평균선 분석 결과, 장기적인 상승 추세가 유지되고 있습니다.")
else:
    print(f"월 단위 이동평균선 분석 결과, 하락 추세로의 전환 가능성이 있습니다.")

# 각 데이터에 대한 시각화
plt.figure(figsize=(14, 10))

# 일 단위 시각화
plt.subplot(3, 1, 1)
plt.plot(daily_df['Date'], daily_df['Close'], label='Close')
plt.plot(daily_df['Date'], daily_df['MA_10'], label='10-Day MA')
plt.plot(daily_df['Date'], daily_df['MA_50'], label='50-Day MA')
plt.title('Daily Close Price and Moving Averages')
plt.legend()

# 주 단위 시각화
plt.subplot(3, 1, 2)
plt.plot(weekly_df.index, weekly_df['Close'], label='Close')
plt.plot(weekly_df.index, weekly_df['MA_10'], label='10-Week MA')
plt.plot(weekly_df.index, weekly_df['MA_50'], label='50-Week MA')
plt.title('Weekly Close Price and Moving Averages')
plt.legend()

# 월 단위 시각화
plt.subplot(3, 1, 3)
plt.plot(monthly_df.index, monthly_df['Close'], label='Close')
plt.plot(monthly_df.index, monthly_df['MA_10'], label='10-Month MA')
plt.plot(monthly_df.index, monthly_df['MA_50'], label='50-Month MA')
plt.title('Monthly Close Price and Moving Averages')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import os

# 저장 디렉토리 확인 및 생성
output_directory = '/content/drive/MyDrive/US MARKET/Data/'
os.makedirs(output_directory, exist_ok=True)


In [ ]:
import yfinance as yf
import pandas as pd

def get_ohlcv_data(ticker, start_date, end_date):
    # Ticker 데이터 다운로드
    data = yf.download(ticker, start=start_date, end=end_date)

    # OHLCV 데이터 반환
    ohlcv_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    return ohlcv_data

def main():
    # 수집할 티커와 기간 설정
    ticker = "META"
    start_date = "2020-01-01"
    end_date = "2024-11-12"

    # OHLCV 데이터 수집
    ohlcv_data = get_ohlcv_data(ticker, start_date, end_date)

    # 데이터 출력
    print(ohlcv_data)

    # 데이터 CSV 파일로 저장
    ohlcv_data.to_csv(f"/content/drive/MyDrive/US MARKET/Data/{ticker}_ohlcv_data.csv")

if __name__ == "__main__":
    main()


In [ ]:
import yfinance as yf
import pandas as pd
from pandas import ExcelWriter

def get_ohlcv_data(ticker, start_date, end_date):
    # Ticker 데이터 다운로드
    data = yf.download(ticker, start=start_date, end=end_date)

    # OHLCV 데이터 반환
    ohlcv_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    return ohlcv_data

def main():
    # 수집할 티커와 기간 설정
    tickers = ["AAPL", "MSFT", "GOOGL","NVDA"]
    start_date = "2020-01-01"
    end_date = "2024-11-13"

    # ExcelWriter를 사용하여 데이터를 Excel 파일에 저장
    with ExcelWriter("/content/drive/MyDrive/US MARKET/Data/ohlcv_data.xlsx") as writer:
        for ticker in tickers:
            # OHLCV 데이터 수집
            ohlcv_data = get_ohlcv_data(ticker, start_date, end_date)

            # 각 티커의 데이터를 별도의 시트로 저장
            ohlcv_data.to_excel(writer, sheet_name=ticker)

            # 데이터 출력
            print(f"{ticker} 데이터:")
            print(ohlcv_data)

if __name__ == "__main__":
    main()


In [ ]:
import yfinance as yf
import pandas as pd
from pandas import ExcelWriter

def get_ohlcv_data(ticker, start_date, end_date):
    # Ticker 데이터 다운로드
    data = yf.download(ticker, start=start_date, end=end_date)

    # OHLCV 데이터 반환
    ohlcv_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Date 컬럼의 timezone 정보 제거 (tz_localize(None) 사용)
    ohlcv_data.reset_index(inplace=True)  # 'Date'를 컬럼으로 변환
    ohlcv_data['Date'] = pd.to_datetime(ohlcv_data['Date']).dt.tz_localize(None)
    ohlcv_data.set_index('Date', inplace=True)  # 'Date'를 다시 인덱스로 설정

    return ohlcv_data

def main():
    # 수집할 티커와 기간 설정
    tickers = ["AAPL", "MSFT", "GOOGL","NVDA","META"]
    start_date = "2020-01-01"
    end_date = "2024-11-13"

    # ExcelWriter를 사용하여 데이터를 Excel 파일에 저장
    with ExcelWriter("/content/drive/MyDrive/US MARKET/Data/ohlcv_data.xlsx") as writer:
        for ticker in tickers:
            # OHLCV 데이터 수집
            ohlcv_data = get_ohlcv_data(ticker, start_date, end_date)

            # 각 티커의 데이터를 별도의 시트로 저장
            ohlcv_data.to_excel(writer, sheet_name=ticker)

            # 데이터 출력
            print(f"{ticker} 데이터:")
            print(ohlcv_data)

if __name__ == "__main__":
    main()